In [48]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers

data = pd.read_csv("Maternal Health Risk Data Set.csv")
label_encoder = LabelEncoder()
data['RiskLevel'] = label_encoder.fit_transform(data['RiskLevel'])
X = data.drop('RiskLevel', axis=1)
y = data['RiskLevel']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

class RBM:
    def __init__(self, n_visible, n_hidden):
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        
        self.weights = np.random.normal(0, 0.01, (n_visible, n_hidden))
        self.visible_bias = np.zeros(n_visible)
        self.hidden_bias = np.zeros(n_hidden)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sample_hidden(self, visible_data):
        hidden_prob = self.sigmoid(np.dot(visible_data, self.weights) + self.hidden_bias)
        hidden_sample = (hidden_prob > np.random.rand(len(visible_data), self.n_hidden)).astype(float)
        return hidden_prob, hidden_sample
    
    def sample_visible(self, hidden_data):
        visible_prob = self.sigmoid(np.dot(hidden_data, self.weights.T) + self.visible_bias)
        visible_sample = (visible_prob > np.random.rand(len(hidden_data), self.n_visible)).astype(float)
        return visible_prob, visible_sample
    
    def contrastive_divergence(self, visible_data, learning_rate=0.1, epochs=10):
        for _ in range(epochs):
            hidden_prob1, hidden_sample1 = self.sample_hidden(visible_data)
            visible_prob, visible_sample = self.sample_visible(hidden_sample1)
            hidden_prob2, hidden_sample2 = self.sample_hidden(visible_sample)
            
            positive_grad = np.dot(visible_data.T, hidden_prob1)
            negative_grad = np.dot(visible_sample.T, hidden_prob2)
            
            self.weights += learning_rate * (positive_grad - negative_grad)
            self.visible_bias += learning_rate * np.mean(visible_data - visible_sample, axis=0)
            self.hidden_bias += learning_rate * np.mean(hidden_prob1 - hidden_prob2, axis=0)
        
        return self.weights, self.hidden_bias

layer_sizes = [X_train.shape[1], 64, 32, 16, 8, 3]
pretrained_weights = []
pretrained_biases = []
current_input = X_train
for i in range(1, len(layer_sizes)):
    rbm = RBM(layer_sizes[i-1], layer_sizes[i])
    weights, bias = rbm.contrastive_divergence(current_input)
    pretrained_weights.append(weights)
    pretrained_biases.append(bias)
    
    _, current_input = rbm.sample_hidden(current_input)

model = keras.Sequential([
    layers.Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(3, activation='softmax')
])
model.layers[0].set_weights([pretrained_weights[0], pretrained_biases[0]])
model.layers[1].set_weights([pretrained_weights[1], pretrained_biases[1]])
model.layers[2].set_weights([pretrained_weights[2], pretrained_biases[2]])
model.layers[3].set_weights([pretrained_weights[3], pretrained_biases[3]])

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)
model.fit(
    X_train, y_train, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2, 
    verbose=0
)

y_pred = np.argmax(model.predict(X_test, verbose=0), axis=-1)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

C:\Users\sench\AppData\Local\Temp\ipykernel_17040\985841073.py:29: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
c:\Users\sench\miniforge3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        76
           1       0.38      1.00      0.55       117
           2       0.00      0.00      0.00       112

    accuracy                           0.38       305
   macro avg       0.13      0.33      0.18       305
weighted avg       0.15      0.38      0.21       305

Confusion Matrix:
 [[  0  76   0]
 [  0 117   0]
 [  0 112   0]]


c:\Users\sench\miniforge3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sench\miniforge3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sench\miniforge3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [51]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import BernoulliRBM
from tensorflow import keras
from tensorflow.keras import layers

data = pd.read_csv("Maternal Health Risk Data Set.csv")
label_encoder = LabelEncoder()
data['RiskLevel'] = label_encoder.fit_transform(data['RiskLevel'])
X = data.drop('RiskLevel', axis=1)
y = data['RiskLevel']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

layer_sizes = [X_train.shape[1], 64, 32, 16, 8, 3]
pretrained_weights = []
pretrained_biases = []

current_input = X_train.copy()
for i in range(1, len(layer_sizes)):
    rbm = BernoulliRBM(
        n_components=layer_sizes[i],
        n_iter=100,
        learning_rate=0.0003,
        batch_size=64,
        random_state=98
    )
    
    binary_input = (current_input > current_input.mean()).astype(int)
    
    rbm.fit(current_input)
    
    pretrained_weights.append(rbm.components_.T)
    pretrained_biases.append(rbm.intercept_hidden_)
    
    current_input = rbm.transform(binary_input)

model = keras.Sequential([
    layers.Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(3, activation='softmax')
])
model.layers[0].set_weights([pretrained_weights[0], pretrained_biases[0]])
model.layers[1].set_weights([pretrained_weights[1], pretrained_biases[1]])
model.layers[2].set_weights([pretrained_weights[2], pretrained_biases[2]])
model.layers[3].set_weights([pretrained_weights[3], pretrained_biases[3]])

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)
model.fit(
    X_train, y_train, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2, 
    verbose=0
)

y_pred = np.argmax(model.predict(X_test, verbose=0), axis=-1)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

c:\Users\sench\miniforge3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        76
           1       0.38      1.00      0.55       117
           2       0.00      0.00      0.00       112

    accuracy                           0.38       305
   macro avg       0.13      0.33      0.18       305
weighted avg       0.15      0.38      0.21       305

Confusion Matrix:
 [[  0  76   0]
 [  0 117   0]
 [  0 112   0]]


c:\Users\sench\miniforge3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sench\miniforge3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sench\miniforge3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
